In [1]:
import numpy as np
import pickle

from pybpl import splines

In [44]:
with open('../substroke_dict.p', 'rb') as fp:
    ss_dict = pickle.load(fp)

In [38]:
def norm_substk(substroke, newscale=105):
    mu = np.mean(substroke, axis=0)
    substroke = substroke - mu
    range_x, range_y = np.ptp(substroke,axis=0)
    scale = newscale / max(1,max(range_x,range_y))
    substroke = substroke*scale
    
    return substroke, mu, scale

In [45]:
a = 0
c = 0
r = 0
s = 0
ss = 1
substroke = ss_dict[a][c][r][s][ss]
print(type(substroke))
print(len(substroke))

<class 'numpy.ndarray'>
49


In [46]:
substroke_n, _, scale = norm_substk(substroke)

In [54]:
def ss_dict_to_spline_dict(ss_dict):
    spline_dict = {}
    n_alpha = len(ss_dict)
    for a in range(n_alpha):
        spline_dict[a] = {}
        alphabet = ss_dict[a]
        n_char = len(alphabet)
        for c in range(n_char):
            spline_dict[a][c] = {}
            char = alphabet[c]
            n_rend = len(char)
            for r in range(n_rend):
                spline_dict[a][c][r] = {}
                rendition = char[r]
                n_stroke = len(rendition)
                for s in range(n_stroke):
                    spline_dict[a][c][r][s] = {}
                    stroke = rendition[s]
                    n_substrokes = len(stroke)
                    for ss in range(n_substrokes):
                        num_steps = len(stroke[ss])
                        if num_steps >= 10:
                            spline_dict[a][c][r][s][ss] = np.zeros((5,2))
                            substk = stroke[ss]
                            substk, _, scale = norm_substk(substk)
                            spline = splines.fit_bspline_to_traj(substk, nland=5)
                            # PyTorch -> Numpy
                            spline = spline.numpy()
                            # Add 2 extra dimensions - scales weighted twice
                            spline = np.append(spline, [[scale, scale]], axis=0)
                            spline_dict[a][c][r][s][ss] = spline
    return spline_dict

In [55]:
spline_dict = ss_dict_to_spline_dict(ss_dict)

/Users/rfeinman/src/pyBPL/pybpl/splines.py:79: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  P, _, _, _ = np.linalg.lstsq(np.matmul(Cof.T, Cof), np.matmul(Cof.T, X))


In [59]:
with open('../spline_dict.p', 'wb') as fp:
    pickle.dump(spline_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)

In [56]:
a = 0
c = 0
r = 0
s = 0
ss = 1
spline = spline_dict[a][c][r][s][ss]
print(spline.shape)

(6, 2)


In [57]:
spline

array([[ 22.07154465,  55.81287384],
       [  4.07091761,  26.45222092],
       [ -2.25516343,   0.58225691],
       [ -9.35992146, -24.93243408],
       [ -8.8428812 , -59.49704742],
       [  2.25875559,   2.25875559]])

In [53]:
np.append(spline, [[0.5, 0.5]], axis=0)

array([[ 116.29638672,  -38.41197205],
       [  98.29576874,  -67.77262878],
       [  91.96968079,  -93.64258575],
       [  84.8649292 , -119.1572876 ],
       [  85.38195801, -153.72189331],
       [   0.5       ,    0.5       ]])